<a href="https://colab.research.google.com/github/sarthak-chakraborty/TF-Lite/blob/master/ModelPersonalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Note:
- After every execution of the tflite convert code and saving it in `custom_keras_model` directory, restart the runtime and execute the import sttements and the base model initialization lines and then the corresponding head model definition.

- The whitelisted operators supported by tflite convert is given in [this link](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/flex/whitelisted_flex_ops.cc)

- Link for the materials: [https://github.com/tensorflow/examples/tree/master/lite/examples/model_personalization](https://github.com/tensorflow/examples/tree/master/lite/examples/model_personalization)

In [1]:
# Clone the github repo
!git clone https://github.com/sarthak-chakraborty/examples.git #(Contains an extra change)

# Actual Repo
# !https://github.com/tensorflow/examples.git

%cd examples/lite/examples/model_personalization/converter
!pip install -e .

Cloning into 'examples'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 10647 (delta 9), reused 29 (delta 0), pack-reused 10601
Receiving objects: 100% (10647/10647), 19.31 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (5446/5446), done.
/content/examples/lite/examples/model_personalization/converter
Obtaining file:///content/examples/lite/examples/model_personalization/converter
     |████████████████████████████████| 86.3MB 64kB/s 
     |████████████████████████████████| 2.1MB 43.0MB/s 
     |████████████████████████████████| 501kB 40.6MB/s 
     |████████████████████████████████| 4.3MB 38.1MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
  Found existing installation: Pillow 7.

In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

In [2]:
"""
Get the Base model
"""

base = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(224,224,3))])
base.save("base_model", save_format="tf")
base = bases.SavedModelBase("base_model")

INFO:tensorflow:Assets written to: base_model/assets


In [3]:
"""
Simple Head Model with only Conv2D and MaxPool2D layers
"""

head = tf.keras.Sequential([
    layers.Conv2D(32, 3, input_shape=(224, 224, 3), padding='same'),
    layers.Activation('relu'),
    layers.MaxPool2D(),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(4, activation='softmax'),
])

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


### TODO: 
#### To execute this piece of code: 
 - Go to `examples/lite/examples/model_personalization/convert/tfltransfer/tflite_transfer_converter.py`
 - In the function `_generate_train_head_model()`, add `converter.allow_custom_ops = True` at line 141.




In [3]:
"""
Mobile Net without BatchNorm
"""

model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), input_shape=(224, 224, 3), padding='same'))
model.add(layers.Activation('relu'))

# (Channels, Strides)
int_layers = [
  (64, (1, 1)),
  (128, (2, 2)),
  (128, (1, 1)),
  (256, (2, 2)),
  (256, (1, 1)),
  (512, (2, 2)),
  *[(512, (1, 1)) for _ in range(5)],
  (1024, (2, 2)),
  (1024, (1, 1))
]
for channels, strides in int_layers:
  # Depthwise
  model.add(layers.DepthwiseConv2D(kernel_size=(3, 3), strides=strides, use_bias=False, padding='same'))
  model.add(layers.Activation('relu'))

  # Pointwise
  model.add(layers.Conv2D(channels, kernel_size=(1, 1), strides=(1, 1), use_bias=False, padding='valid'))
  model.add(layers.Activation('relu'))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(4, activation='softmax'))

head = model


# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')

"""
On adding the line `converter.allow_custom_ops=True` at the desired location, it will get converted, since the converter expects that a custom operation is there

While training on android device the following error shows up:

Caused by: java.lang.IllegalArgumentException: Internal error: Failed to run on the given Interpreter: Encountered unresolved custom op: DepthwiseConv2dNativeBackpropInput.
  Node number 146 (DepthwiseConv2dNativeBackpropInput) failed to prepare.
"""

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


ConverterError: ignored

In [3]:
"""
BatchNorm Layer
"""

model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), input_shape=(224, 224, 3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(4, activation='softmax'))

head = model


# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')


Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

AttributeError: ignored

In [3]:
"""
Skip Connection using Functional API of tf.keras
However, only Sequential API can be converted
"""

x_in = tf.keras.Input(shape=(224,224,3))
x_shortcut = x_in

x = layers.Conv2D(3, kernel_size=(1,1), strides=(1,1), padding='valid')(x_in)
x = layers.Activation('relu')(x)
x = layers.Conv2D(3, kernel_size=(3,3), strides=(1,1), padding='same')(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(3, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
x = layers.Activation('relu')(x)

x = layers.Add()([x, x_shortcut])
x_out = layers.Activation('relu')(x)

model = tf.keras.Model(inputs=x_in, outputs=x_out)

head = model


# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in

RuntimeError: ignored